In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [2]:
# cargamos el dataframe creado en la lección anterior
df = pd.read_csv("archivos_pair/world_data_full_apply.csv",index_col=0)
df.head(1)

,country,density,abbreviation,agricultural_land,land_area,armed_forces_size,birth_rate,calling_code,capital/major_city,co2-emissions,cpi,cpi_change,currency-code,fertility_rate,forested_area,gasoline_price,gdp,gross_primary_education_enrollment,gross_tertiary_education_enrollment,infant_mortality,largest_city,life_expectancy,maternal_mortality_ratio,minimum_wage,official_language,out_of_pocket_health_expenditure,physicians_per_thousand,population,population_labor_force_participation,tax_revenue,total_tax_rate,unemployment_rate,urban_population,latitude,longitude,continent
0,Afghanistan,60.0,AF,58.1,652.23,323.0,32.49,93.0,Kabul,8.672,149.9,2.3,AFN,4.47,2.1,0.7,1.910135e+10,104.0,9.7,47.9,Kabul,64.5,638.0,0.43,Pashto,78.4,0.28,NaN,48.9,9.3,71.4,11.12,NaN,33.93911,67.709953,Asia


In [3]:
# Obtenemos la lista de columnas categóricas que tienen nulos
nulos_esta_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['abbreviation', 'capital/major_city', 'currency-code', 'largest_city',
       'official_language', 'continent'],
      dtype='object')


In [14]:
# Obtenemos la lista de columnas numericas que tienen nulos
nulos_esta_num = df[df.columns[df.isnull().any()]].select_dtypes(include = np.number).columns
print("Las columnas numéricas que tienen nulos son : \n ")
print(nulos_esta_num)

Las columnas numéricas que tienen nulos son : 
 
Index(['agricultural_land', 'land_area', 'armed_forces_size', 'birth_rate',
       'calling_code', 'co2-emissions', 'cpi', 'cpi_change', 'fertility_rate',
       'forested_area', 'gasoline_price', 'gdp',
       'gross_primary_education_enrollment',
       'gross_tertiary_education_enrollment', 'infant_mortality',
       'life_expectancy', 'maternal_mortality_ratio', 'minimum_wage',
       'out_of_pocket_health_expenditure', 'physicians_per_thousand',
       'population', 'population_labor_force_participation', 'tax_revenue',
       'total_tax_rate', 'unemployment_rate', 'urban_population', 'latitude',
       'longitude'],
      dtype='object')


In [6]:
# lo primero que vamos a hacer es calcular el número de nulos que tenemos en cada una de las columnas numericas
nulos_num=df[nulos_esta_num].isnull().sum() / df.shape[0]*100

In [7]:
# lo convertimos a DataFrame
df_nulos = pd.DataFrame(nulos_num, columns = ["%_nulos"])

# filtramos el DataFrame para quedarnos solo con aquellas columnas que tengan nulos
df_nulos[df_nulos["%_nulos"] > 0]

,%_nulos
agricultural_land,3.589744
land_area,15.384615
armed_forces_size,14.871795
birth_rate,3.076923
calling_code,0.512821
co2-emissions,6.153846
cpi,10.256410
cpi_change,8.205128
fertility_rate,3.589744
forested_area,3.589744


In [5]:
lista_columnas_num=['agricultural_land', 'land_area', 'armed_forces_size', 'birth_rate',
       'calling_code', 'co2-emissions', 'cpi', 'cpi_change', 'fertility_rate',
       'forested_area', 'gasoline_price', 'gdp',
       'gross_primary_education_enrollment',
       'gross_tertiary_education_enrollment', 'infant_mortality',
       'life_expectancy', 'maternal_mortality_ratio', 'minimum_wage',
       'out_of_pocket_health_expenditure', 'physicians_per_thousand',
       'population', 'population_labor_force_participation', 'tax_revenue',
       'total_tax_rate', 'unemployment_rate', 'urban_population', 'latitude',
       'longitude']
      

In [11]:
df.sample(5)

,country,density,abbreviation,agricultural_land,land_area,armed_forces_size,birth_rate,calling_code,capital/major_city,co2-emissions,cpi,cpi_change,currency-code,fertility_rate,forested_area,gasoline_price,gdp,gross_primary_education_enrollment,gross_tertiary_education_enrollment,infant_mortality,largest_city,life_expectancy,maternal_mortality_ratio,minimum_wage,official_language,out_of_pocket_health_expenditure,physicians_per_thousand,population,population_labor_force_participation,tax_revenue,total_tax_rate,unemployment_rate,urban_population,latitude,longitude,continent,agricultural_landX,land_areaX,birth_rateX,co2-emissionsX,cpiX,fertility_rateX,forested_areaX,gdpX,gross_primary_education_enrollmentX,gross_tertiary_education_enrollmentX,life_expectancyX,out_of_pocket_health_expenditureX,physicians_per_thousandX,population_labor_force_participationX,tax_revenueX
140,Portugal,111.0,PT,39.5,92.212,52.0,8.50,351.0,Lisbon,48.742,110.62,0.3,EUR,1.38,34.6,1.54,2.376861e+11,106.2,63.9,3.1,Lisbon,81.3,8.0,3.78,Portuguese,27.7,5.12,NaN,58.8,22.8,39.8,6.33,NaN,39.399872,-8.224454,Europe,39.5,92.212,8.50,48.742,110.62,1.38,34.6,2.376861e+11,106.2,63.9,81.3,27.7,5.12,58.8,22.8
180,Turkmenistan,13.0,TM,72.0,488.100,42.0,23.83,993.0,Ashgabat,70.630,NaN,NaN,TMT,2.79,8.8,0.29,4.076114e+10,88.4,8.0,39.3,Ashgabat,68.1,7.0,0.88,Turkmen,71.1,2.22,NaN,64.5,NaN,NaN,3.91,NaN,38.969719,59.556278,Asia,72.0,488.100,23.83,70.630,124.74,2.79,8.8,4.076114e+10,88.4,8.0,68.1,71.1,2.22,64.5,16.3
49,Dominican Republic,225.0,DO,48.7,48.670,71.0,19.51,1.0,Santo Domingo,25.258,135.50,1.8,DOP,2.35,41.7,1.07,8.894130e+10,105.7,59.9,24.1,Santo Domingo,73.9,95.0,0.40,Spanish,43.7,1.56,NaN,64.3,13.0,48.8,5.84,NaN,18.735693,-70.162651,Central America,48.7,48.670,19.51,25.258,135.50,2.35,41.7,8.894130e+10,105.7,59.9,73.9,43.7,1.56,64.3,13.0
146,Saint Lucia,301.0,LC,17.4,616.000,NaN,12.00,1.0,Castries,414.000,110.13,1.9,XCD,1.44,33.2,1.30,2.122451e+09,102.6,14.1,14.9,Castries,76.1,117.0,NaN,English,48.4,0.64,182.790,67.1,18.2,34.7,20.71,34.280,13.909444,-60.978893,Central America,17.4,616.000,12.00,414.000,110.13,1.44,33.2,2.122451e+09,102.6,14.1,76.1,48.4,0.64,67.1,18.2
29,Cape Verde,138.0,CV,19.6,4.033,1.0,19.49,238.0,Praia,543.000,110.50,1.1,CVE,2.27,22.5,1.02,1.981846e+09,104.0,23.6,16.7,Praia,72.8,58.0,0.68,Portuguese,23.2,0.77,483.628,60.5,20.1,37.5,12.25,364.029,16.538800,-23.041800,Africa,19.6,4.033,19.49,543.000,110.50,2.27,22.5,1.981846e+09,104.0,23.6,72.8,23.2,0.77,60.5,20.1


In [ ]:
lista_mediana = ['agricultural_land', 'land_area', 'birth_rate', 'co2-emissions', 'cpi',"fertility_rate","forested_area", 'gdp', 'gross_primary_education_enrollment', 'gross_tertiary_education_enrollment', 'life_expectancy', 'out_of_pocket_health_expenditure', 'physicians_per_thousand', 'population_labor_force_participation', 'tax_revenue']

lista_knni = ['armed_forces_size', 'cpi_change',"gasoline_price",'infant_mortality', 'maternal_mortality_ratio', 'minimum_wage', 'population', 'unemployment_rate', 'urban_population']
lista_moda = ['calling_code']

In [8]:
imputer = SimpleImputer(strategy='median')

In [9]:


imputed= imputer.fit_transform(df[['agricultural_land', 'land_area', 'birth_rate', 'co2-emissions', 'cpi',"fertility_rate","forested_area", 'gdp', 'gross_primary_education_enrollment', 'gross_tertiary_education_enrollment', 'life_expectancy', 'out_of_pocket_health_expenditure', 'physicians_per_thousand', 'population_labor_force_participation', 'tax_revenue']])  # Transformamos correctamente en 2D


In [10]:
df[['agricultural_landX', 'land_areaX', 'birth_rateX', 'co2-emissionsX', 'cpiX',"fertility_rateX","forested_areaX", 'gdpX', 'gross_primary_education_enrollmentX', 'gross_tertiary_education_enrollmentX', 'life_expectancyX', 'out_of_pocket_health_expenditureX', 'physicians_per_thousandX', 'population_labor_force_participationX', 'tax_revenueX']]=imputed

In [19]:
df=df.drop(lista_mediana,axis=1)

In [20]:
df.head(3)

,country,density,abbreviation,armed_forces_size,calling_code,capital/major_city,cpi_change,currency-code,gasoline_price,infant_mortality,largest_city,maternal_mortality_ratio,minimum_wage,official_language,population,total_tax_rate,unemployment_rate,urban_population,latitude,longitude,continent,agricultural_landX,land_areaX,birth_rateX,co2-emissionsX,cpiX,fertility_rateX,forested_areaX,gdpX,gross_primary_education_enrollmentX,gross_tertiary_education_enrollmentX,life_expectancyX,out_of_pocket_health_expenditureX,physicians_per_thousandX,population_labor_force_participationX,tax_revenueX
0,Afghanistan,60.0,AF,323.0,93.0,Kabul,2.3,AFN,0.70,47.9,Kabul,638.0,0.43,Pashto,NaN,71.4,11.12,NaN,33.939110,67.709953,Asia,58.1,652.230,32.49,8.672,149.90,4.47,2.1,1.910135e+10,104.0,9.7,64.5,78.4,0.28,48.9,9.3
1,Albania,105.0,AL,9.0,355.0,Tirana,1.4,ALL,1.36,7.8,Tirana,15.0,1.12,Albanian,NaN,36.6,12.33,NaN,41.153332,20.168331,Europe,43.1,28.748,11.78,4.536,119.05,1.62,28.1,1.527808e+10,107.0,55.0,78.5,56.9,1.20,55.7,18.6
2,Algeria,18.0,DZ,317.0,213.0,Algiers,2.0,DZD,0.28,20.1,Algiers,112.0,0.95,Arabic,NaN,66.1,11.70,NaN,28.033886,1.659626,Africa,17.4,148.460,24.28,150.006,151.36,3.02,0.8,1.699882e+11,109.9,51.4,76.7,28.1,1.72,41.2,37.2


In [18]:
df.describe(include=np.number).T

,count,mean,std,min,25%,50%,75%,max
density,195.0,1.345045e+02,1.573068e+02,1.265000e+00,2.616850e+01,8.300000e+01,1.655000e+02,8.470000e+02
agricultural_land,188.0,3.911755e+01,2.178305e+01,6.000000e-01,2.170000e+01,3.960000e+01,5.537500e+01,8.260000e+01
land_area,165.0,2.552127e+02,2.583850e+02,0.000000e+00,4.127700e+01,1.484600e+02,4.300000e+02,9.640000e+02
armed_forces_size,166.0,1.037831e+02,1.671580e+02,0.000000e+00,1.025000e+01,2.850000e+01,1.282500e+02,9.360000e+02
birth_rate,189.0,2.021497e+01,9.945774e+00,5.900000e+00,1.130000e+01,1.795000e+01,2.875000e+01,4.608000e+01
calling_code,194.0,3.605464e+02,3.232364e+02,1.000000e+00,8.250000e+01,2.555000e+02,5.067500e+02,1.876000e+03
co2-emissions,183.0,1.259155e+02,1.817949e+02,1.016000e+00,7.600000e+00,3.447700e+01,1.850305e+02,7.279730e+02
cpi,175.0,1.441936e+02,5.478400e+01,9.903000e+01,1.136700e+02,1.247400e+02,1.559250e+02,5.509300e+02
cpi_change,179.0,6.722346e+00,2.445041e+01,-4.300000e+00,1.000000e+00,2.300000e+00,4.250000e+00,2.549000e+02
fertility_rate,188.0,2.698138e+00,1.282267e+00,9.800000e-01,1.705000e+00,2.245000e+00,3.597500e+00,6.910000e+00


In [ ]:
for col in nulos_esta_num:
    print(f'para la columa {col}')
    display(df[col].value_counts() / df.shape[0] * 100)

 Reemplazar con estadísticos (media, moda o mediana), para lo que utilizaremos el .fillna() que hemos aprendido o utilizar el SimpleImputer.

Reemplazar con modelos estadísticos usando IterativeImputer o KNNImputer(gran camntidad de valores faltantes)